In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from PIL import Image

from core.integrate import FeatureFusionScalableTSDFVolume
from core.dataset import ScanNet
from core.labeler import CLIPTextQuerier, KMeansLabeler

In [3]:
scene = "scannet_scene0000_00"
load_dir = "/home/quanta/Experiments/feature-instance-fusion/" + scene + "/"

In [4]:
device = "cpu"
tsdf_volume_unpruned = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=device,
)
tsdf_volume_unpruned.load(load_dir + "tsdf/tsdf_volume_unpruned.pt")
tsdf_volume_unpruned.load_feats(load_dir + "tsdf_feature_lseg/feats.pt")

tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=device,
)
tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")

In [5]:
torch.isin(tsdf_volume._voxel_hash, tsdf_volume_unpruned._voxel_hash).float().min()

tensor(1.)

In [6]:
unique_hash, indices = torch.cat([tsdf_volume_unpruned._voxel_hash, tsdf_volume._voxel_hash]).unique(sorted=True, return_inverse=True)

In [7]:
(indices[: tsdf_volume_unpruned.num_voxel] == torch.arange(tsdf_volume_unpruned.num_voxel, device=device)).float().min()

tensor(1.)

In [8]:
tsdf_volume.reset_feature(dim=512, include_var=False, dtype=torch.float32)

In [9]:
tsdf_volume._feat = tsdf_volume_unpruned._feat[indices]
tsdf_volume._feat_w_sum = tsdf_volume_unpruned._feat_w_sum[indices]

In [10]:
save_dir = load_dir + "tsdf_feature_lseg_pruned" + "/"
print(save_dir)

/home/quanta/Experiments/feature-instance-fusion/scannet_scene0000_00/tsdf_feature_lseg_pruned/


In [11]:
tsdf_volume.save_feats(save_dir + "feats.pt")